In [41]:
import argparse
import logging
import matplotlib.pyplot as plt
from multiprocess import Pool
import numpy as np
import pandas as pd
import random
import sys
import operator
import math
import graphviz


from sklearn.datasets import load_boston, load_breast_cancer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.linear_model import Ridge
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils._testing import assert_equal, assert_almost_equal
from sklearn.utils.validation import check_random_state

from gplearn.genetic import SymbolicClassifier, SymbolicRegressor
from gplearn.genetic import SymbolicTransformer
from gplearn.functions import make_function

# Graphs and dataset
import matplotlib.pyplot as plt
import seaborn as sb
from pmlb import fetch_data

# Target formula:
x0 = np.arange(-1, 1, 1/10.)
x1 = np.arange(-1, 1, 1/10.)
x0, x1 = np.meshgrid(x0, x1)
y_truth = x0**2 - x1**2 + x1 - 1

# SCOOP
#toolbox.register("map", futures.map)

# Best configurations as commandlines (first number is the configuration ID; same order as above):
# 45   --pop=438 --cros=0.53 --mut=0.13
# 104  --pop=438 --cros=0.53 --mut=0.19 based on irace

def main(POP, CXPB, MUTPB):
    
    # line =plt.plot(graph)
    # plt.show()

    # line =plt.plot(num_features)
    # plt.show()
    
    rng = check_random_state(0)

    # Training samples
    X_train = rng.uniform(-1, 1, 100).reshape(50, 2)
    y_train = X_train[:, 0]**2 - X_train[:, 1]**2 + X_train[:, 1] - 1

    # Testing samples
    X_test = rng.uniform(-1, 1, 100).reshape(50, 2)
    y_test = X_test[:, 0]**2 - X_test[:, 1]**2 + X_test[:, 1] - 1
    
    # To ensure CXPB + MUTPB are 1 or less than
    Prob = MUTPB + CXPB
    if Prob > 1:
        MUTPB = 1 - CXPB
    
    est_gp = SymbolicRegressor(population_size=POP,
                           generations=20, stopping_criteria=0.01,
                           p_crossover=CXPB, p_subtree_mutation=MUTPB,
                           p_hoist_mutation=0, p_point_mutation=0,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0)
                           
    est_gp.fit(X_train, y_train)

    idx = est_gp._program.parents['donor_idx']
    fade_nodes = est_gp._program.parents['donor_nodes']
    dot_data = est_gp._programs[-2][idx].export_graphviz(fade_nodes=fade_nodes)
    graph2 = graphviz.Source(dot_data)
    graph2.view()

if __name__ == "__main__":
    main(438,0.53,0.13)
    

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    37.42          411.735       31         0.607299         0.758801      7.53s
   1    11.09          1.78534       11          0.43086          0.47412      5.48s
   2     9.78          5.67468       31         0.373196         0.541958      3.87s
   3    11.70         0.983537       11         0.267983         0.170448      4.53s
   4     9.93          1.22594       11         0.242259         0.401965      4.36s
   5     7.82         0.876977       11         0.241677         0.407205      3.56s
   6    10.34         0.765584       11        0.0686481        0.0686481      2.63s
   7     9.67         0.743111       11        0.0686481        0.0686481      2.34s
   8     9.29          4.99423       11        0.0686481        0.0686481  

IndentationError: expected an indented block (<ipython-input-26-685334f241bf>, line 11)